In [6]:
from pyspark.sql.types import IntegerType
from typedspark import Column, Schema
from typedspark._core.datatypes import StructType

class A(Schema):
    a: Column[IntegerType]

class B(Schema):
    a: Column[IntegerType]
    b: Column[StructType[A]]

B.b.dtype.schema


from pyspark.sql.types import IntegerType

from typedspark import Column, Schema


class A(Schema):
    a: Column[IntegerType]